<a href="https://colab.research.google.com/github/nilsroemertum/synlab-slim-viewer/blob/master/notebooks/viewers_deployment/dev_deploy_gc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gcpProjectID = 'synlab-pacs-workflow'
gcpClientID = '"62039621046-rt1fu02rnnfrlf2icjh2q1cmervka8au.apps.googleusercontent.com"'
FirebaseProjectName = 'synlab-pacs-slim-viewer'

!npm install -g yarn
!curl -sL https://firebase.tools | bash
!git clone https://github.com/nilsroemertum/synlab-slim-viewer.git
!npm install -g n
!n install 24 && n 24
!cd synlab-slim-viewer && yarn

settings = """
window.config = {
  // This must match the location configured for web server
  path: '/',
  servers: [
    {
      id: 'example',
      // This must match the proxy location configured for the DICOMweb server
      url:"https://proxy.imaging.datacommons.cancer.gov/current/viewer-only-no-downloads-see-tinyurl-dot-com-slash-3j3d9jyp/dicomWeb",
      write: false
    }
  ],
 oidc: {
    authority: "https://accounts.google.com",
    clientId: %s,
    scope: "email profile openid https://www.googleapis.com/auth/cloud-healthcare",
    grantType: "implicit"
  },
  disableWorklist: false,
  disableAnnotationTools: false,
  enableServerSelection: true,
  mode: 'light',
  preload: true,
  annotations: [
    {
      finding: {
        value: '85756007',
        schemeDesignator: 'SCT',
        meaning: 'Tissue'
      },
      findingCategory: {
        value: '91723000',
        schemeDesignator: 'SCT',
        meaning: 'Anatomical structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 255, 0, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      },
    },
    {
      finding: {
        value: '108369006',
        schemeDesignator: 'SCT',
        meaning: 'Tumor'
      },
      findingCategory: {
        value: '49755003',
        schemeDesignator: 'SCT',
        meaning: 'Morphologically abnormal structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 0, 255, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '34823008',
        schemeDesignator: 'SCT',
        meaning: 'Tumor necrosis'
      },
      findingCategory: {
        value: '49755003',
        schemeDesignator: 'SCT',
        meaning: 'Morphologically abnormal structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [51, 204, 51, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '369705002',
        schemeDesignator: 'SCT',
        meaning: 'Invasive tumor border'
      },
      findingCategory: {
        value: '395557000',
        schemeDesignator: 'SCT',
        meaning: 'Tumor finding'
      },
      geometryTypes: ['line', 'freehandline'],
      style: {
        stroke: {
          color: [51, 102, 255, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '399721002',
        schemeDesignator: 'SCT',
        meaning: 'Tumor infiltration by lymphocytes present'
      },
      findingCategory: {
        value: '395557000',
        schemeDesignator: 'SCT',
        meaning: 'Tumor finding'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [51, 204, 204, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '47973001',
        schemeDesignator: 'SCT',
        meaning: 'Artifact'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 80, 80, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    }
  ]
}
"""%(gcpClientID)

with open("synlab-slim-viewer/public/config/example.js", "w") as f:
    f.write(settings)

settings_firebase = """
{
  "hosting": {
    "public": "build",
    "ignore": [
      "firebase.json",
      "**/.*",
      "**/node_modules/**"
    ],
    "rewrites": [
      {
        "source": "**",
        "destination": "/index.html"
      }
    ]
  }
}
"""

with open("synlab-slim-viewer/firebase.json", "w") as f:
  f.write(settings_firebase)

!cd synlab-slim-viewer && REACT_APP_CONFIG=example PUBLIC_URL='/' yarn build
!firebase login --no-localhost
!gcloud auth login
!gcloud config set project $gcpProjectID
!gcloud config get project
!firebase projects:list
!cd synlab-slim-viewer && firebase deploy
